# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [5]:
# Create our features
X = df.drop(columns="loan_status")

# Create our target
y = df["loan_status"].copy()
y.head()

0    low_risk
1    low_risk
2    low_risk
3    low_risk
4    low_risk
Name: loan_status, dtype: object

In [6]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.390633,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [7]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [8]:
# Split the X and y into X_train, X_test, y_train, y_test
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=1, 
                                                   stratify=y)
X_train.shape

(51612, 85)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [9]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
39692,10000.0,0.1131,328.86,MORTGAGE,49000.0,Source Verified,Jan-2019,n,24.54,0.0,...,100.0,0.0,1.0,0.0,309140.0,56262.0,9500.0,54197.0,N,N
37830,21000.0,0.1072,684.74,RENT,57400.0,Not Verified,Jan-2019,n,34.52,0.0,...,100.0,0.0,0.0,0.0,98100.0,35707.0,64700.0,30000.0,N,N
64531,35000.0,0.0881,1109.90,OWN,180000.0,Not Verified,Jan-2019,n,3.01,0.0,...,94.7,0.0,0.0,0.0,327877.0,1222.0,41500.0,26177.0,N,N
28730,20000.0,0.0819,628.49,OWN,80000.0,Source Verified,Feb-2019,n,26.51,0.0,...,90.0,0.0,0.0,0.0,143541.0,81162.0,14500.0,111641.0,N,N
35102,12600.0,0.1356,427.96,MORTGAGE,71000.0,Source Verified,Feb-2019,n,22.38,1.0,...,88.9,0.0,0.0,0.0,316682.0,33433.0,26500.0,50182.0,N,N


In [10]:
#have to turn home_ownership to a float rather then "mortgage" or "own
from sklearn.preprocessing import LabelEncoder
def convert_str_to_float(column_name,df):
    label_encoder = LabelEncoder()
    label_encoder.fit(df[column_name])
    df[column_name] = label_encoder.transform(df[column_name])
    return df
#This is for X_train
X_train=convert_str_to_float('home_ownership',X_train)
X_train=convert_str_to_float('verification_status',X_train)
X_train=convert_str_to_float('pymnt_plan',X_train)
X_train=convert_str_to_float('hardship_flag',X_train)
X_train=convert_str_to_float('debt_settlement_flag',X_train)
X_train=convert_str_to_float('application_type',X_train)
X_train=convert_str_to_float('initial_list_status',X_train)

#This is for X_test
X_test=convert_str_to_float('home_ownership',X_test)
X_test=convert_str_to_float('verification_status',X_test)
X_test=convert_str_to_float('pymnt_plan',X_test)
X_test=convert_str_to_float('hardship_flag',X_test)
X_test=convert_str_to_float('debt_settlement_flag',X_test)
X_test=convert_str_to_float('application_type',X_test)
X_test=convert_str_to_float('initial_list_status',X_test)


months_num = {
    "Jan-2019": 1,
    "Feb-2019": 2,
    "Mar-2019": 3,
    "Apr-2019": 4,
    "May-2019": 5,
    "Jun-2019": 6,
    "Jul-2019": 7,
    "Aug-2019": 8,
    "Sep-2019": 9,
    "Oct-2019": 10,
    "Nov-2019": 11,
    "Dec-2019": 12,
}

#have to change months to numbers - this is clearly not the best way of doing it as it assumes it is all in the same year however I'm not to sure how to do it
X_train["issue_d"] = X_train["issue_d"].apply(lambda x: months_num[x])
X_test["issue_d"] = X_test["issue_d"].apply(lambda x: months_num[x])
X_train["next_pymnt_d"] = X_train["next_pymnt_d"].apply(lambda x: months_num[x])
X_test["next_pymnt_d"] = X_test["next_pymnt_d"].apply(lambda x: months_num[x])


In [11]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [12]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [13]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)
rf_model = rf_model.fit(X_train_scaled, y_train)
y_pred = rf_model.predict(X_test_scaled)

In [14]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.6605982410126868

In [15]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   28,    59],
       [   11, 17107]])

In [16]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.72      0.32      1.00      0.44      0.57      0.30        87
   low_risk       1.00      1.00      0.32      1.00      0.57      0.34     17118

avg / total       1.00      1.00      0.33      1.00      0.57      0.34     17205



In [17]:
# List the features sorted in descending order by feature importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.07545697499682684, 'last_pymnt_amnt'),
 (0.07355862339086773, 'total_rec_int'),
 (0.07242564280515527, 'total_rec_prncp'),
 (0.06485639210814508, 'total_pymnt'),
 (0.061299894478688076, 'total_pymnt_inv'),
 (0.020161306672556282, 'installment'),
 (0.019804307185450655, 'out_prncp_inv'),
 (0.019792957264954926, 'out_prncp'),
 (0.01734548585349599, 'mo_sin_old_rev_tl_op'),
 (0.017091349666116664, 'dti'),
 (0.01650803274645083, 'mo_sin_old_il_acct'),
 (0.01495247702939957, 'loan_amnt'),
 (0.01483922379173835, 'max_bal_bc'),
 (0.01481393804285919, 'bc_open_to_buy'),
 (0.014609408733482416, 'issue_d'),
 (0.014461961358627118, 'bc_util'),
 (0.014068988704777923, 'total_bal_ex_mort'),
 (0.013804628129694347, 'tot_cur_bal'),
 (0.013761634946569384, 'tot_hi_cred_lim'),
 (0.013620355116461773, 'revol_bal'),
 (0.013490869161152969, 'annual_inc'),
 (0.013457336717500589, 'total_rec_late_fee'),
 (0.013107025003602725, 'avg_cur_bal'),
 (0.012958556765482193, 'total_il_high_credit_limit'),
 (0.01

### Easy Ensemble Classifier

In [18]:
#Not to sure how this works


### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.